### Modules and data

In [1]:
# Libraries and data loading
import matplotlib.pyplot as plt
import numpy as np
import gurobipy as gb
import os
from pickle import dump, load
from time import process_time
from copy import deepcopy
from source import *
#from multiprocess import pool
from scipy.spatial.distance import euclidean
from scipy import stats

path = os.getcwd()

# Load data
file = open(path+"/Multiobjective/Configurations/Open Stations/open_stations_0.0","rb"); S = load(file); file.close()
K_s = {}                # {scenario:{station:[vehicles]}}n",
K = {}                  # {scenario:[vehicles]}n",
Ksuperprime = {}

for sc in range(25):
    file = open(path+f"/Data/K/K_sc{sc}", "rb")
    K[sc] = load(file)
    Ksuperprime[sc] = deepcopy(K[sc])
    file.close()

    file = open(path+f"/Data/K_s10/Ks_sc{sc}", "rb")
    K_s[sc] = load(file); file.close()

original_stress = {}
for s in S:
    original_stress[s] = np.mean(np.array([len(K_s[sc][s]) for sc in range(25)]))

### Auxiliary functions

In [2]:
# Auxiliary functions - Number of chargers
def generate_routes_given_number_chargers_individual(s,S,y,K,K_s_prime,max_chargers=8,exhaust=False):

    total_serviced_costumers:dict[list[list]] = dict()
    routes = dict()


    for sc in range(25):
        K, _, S_k, a, t = load_pickle(path+"/Data/",sc)
        K_s = K_s_prime[sc]

        routes[sc] = list()

        S_k = {k:[ss for ss in S_k[k] if ss in S] for k in K_s[s]}
        a.update({("s",s):0}); a.update({("e",s):40})
        t.update({("s",s):0}); t.update({("e",s):0})
    
        mp = gb.Model("Restricted Master Problem")
    
        dummy_0 = {k:mp.addVar(vtype=gb.GRB.CONTINUOUS, obj=1, name=f"st0_{k}") for k in K_s[s]}
        aux = mp.addVar(vtype=gb.GRB.CONTINUOUS, obj=2, name=f"aux_{s}")
    
        vehic_assign = {}
        for k in K_s[s]:
            vehic_assign[k] = mp.addLConstr(dummy_0[k] == 1, name=f"V{k}_assignment")
    
        st_conv = mp.addConstr(aux <= y, f"S{s}_convexity")
    
        mp.setParam("OutputFlag",0)
    
        time0 = process_time(); i = 0
        lbd = []; lastMP = 1e3
    
        while True:
            
            mp.update()
            mp.optimize()
    
            pi = {k:vehic_assign[k].getAttr("Pi") for k in K_s[s]}
            sigma = st_conv.getAttr("Pi")
            thisMP = mp.getObjective().getValue()
    
            if (lastMP-thisMP < 0.1 and thisMP > 5) or (lastMP-thisMP < 0.01 and thisMP<=5):
                #if y == 8: print(f"ttOut of Labeling scenario {sc}")
                mpsol = thisMP; break
    
            #print(f"tIteration {i}:ttMP obj: {round(mp.getObjective().getValue(),2)}ttime: {round(process_time()-time0,2)}s")
            i += 1
    
            V,A,rc = get_graph_chargers(s,K_s[s],a,pi,sigma)
            ext = vertices_extensions(V,A)
            
            #if y == 8:
            #    print(f"ttIteration {i}. Labeling for scenario {sc}")
    
            routes_DFS = {"s":([0],0)}; covered_nodes = {v:0 for v in V}; best_len = {v:0 for v in V}; best_t = {v:0 for v in V}; best_h = {v:30 for v in V}; num_improv = {v:0 for v in V}; tried = {v:0 for v in V}
            marked = {}; max_successors = {v:100 for v in V}
            mod_label_DFS(v="s",rP=0,tP=0,qP=0,hP=0,P=[],cK=covered_nodes,L=routes_DFS,s=s,r=rc,t=t,a=a,ext=ext,best_len=best_len,best_t=best_t,best_h=best_h,tried=tried,marked=marked,num_improv=num_improv,max_successors=max_successors)
            del routes_DFS["s"]
            
            opt = 0
            for l in routes_DFS:
                if routes_DFS[l][1] < -0.001:
                    col = {k:1 if k in routes_DFS[l][0] else 0 for k in K_s[s]}
                    new_Col = gb.Column(col.values(),vehic_assign.values())
                    lbd.append(mp.addVar(vtype=gb.GRB.CONTINUOUS,obj=0,column=new_Col))
    
                    mp.chgCoeff(st_conv,lbd[-1],1)
                    opt +=1
            if i == 1:
                original_routes = routes_DFS
    
            if opt == 0 : mpsol = thisMP; break
            lastMP = thisMP
        
        for v in mp.getVars():
            v.vtype = gb.GRB.BINARY
        #if mpsol < len(K_s[s]): mp.setParam("MIPGap",0.01)
        mp.setParam("MIPFocus",2)
        mp.update(); mp.optimize()
    
        # print(f"ttSolved IMP scenario {sc}, with {mp.getObjective().getValue()} unnasigned vehicles")
        # print(f"IMP solution: {mp.getObjective().getValue()} assigned vehicles with {y} chargers")
        for l in lbd:
            if l.X > 0.5:
                col = [k for k in K_s[s] if mp.getCoeff(vehic_assign[k],l) == 1]
                new_col = sorted(col, key = lambda ii: a.get((ii,s)))
                routes[sc].append(new_col)
        
    
        total_serviced_costumers[sc]:list = list()
    
        for route in routes[sc]:
            total_serviced_costumers[sc].extend(route)
        
        if len(total_serviced_costumers[sc]) < len(K_s_prime[sc][s]) and y < max_chargers:
            # print(f't{y} number of chargers is infeasible')
            return None, None
    
    return routes, total_serviced_costumers


def generate_routes_given_number_chargers(S,y,K_prime,K_s_prime,exhaust=False):

    total_serviced_costumers:dict[dict[list]] = dict()

    sigmass = dict()
    sigmass.update({s:0 for s in S})

    missing = False
    
    for sc in range(25):
        routes = list()
        _, _, S_k, a, t = load_pickle(path+"/Data/",sc)
        for s in S:
            a.update({("s",s):0}); a.update({("e",s):40})
            t.update({("s",s):0}); t.update({("e",s):0})

        K = K_prime[sc]
        K_s = K_s_prime[sc]
    
        routes, sigmas = second_stage_chargers(S, K, K_s, y, a, t, sc)
        sigmass[s] += sigmas[s]/25

        total_serviced_costumers[sc]:dict = dict()
        total_total:list = list()
        for s in S:
            total_serviced_costumers[sc][s]:list = list()
            for route in routes[s]:
                total_serviced_costumers[sc][s].extend(route)
                total_total.extend(route)

        # for s in S:
        #     for k in K_s_prime[sc][s]:
        #         if k not in total_total and y[s] < 8:
        #             missing = True
        #             sigmass = sigmas
        #             break
        #     if missing:
        #         break
            
        # if missing:
        #     break
        
    return routes, total_serviced_costumers, sigmass, missing


def generate_initial_routes(S, y, K_prime, K_s_prime, exhaust=False):

    total_serviced_costumers:dict[dict[list]] = dict()

    sigmass = dict()
    sigmass.update({s:0 for s in S})

    def generate_routes_for_scenario(sc):
        routes = list()
        _, _, S_k, a, t = load_pickle(path+"/Data/",sc)
        for s in S:
            a.update({("s",s):0}); a.update({("e",s):40})
            t.update({("s",s):0}); t.update({("e",s):0})

        K = K_prime[sc]
        K_s = K_s_prime[sc]
    
        routes, sigmas = second_stage_chargers(S, K, K_s, y, a, t, sc)
        sigmass[s] += sigmas[s]/25

        total_serviced_costumers[sc]:dict = dict()
        total_total:list = list()
        for s in S:
            total_serviced_costumers[sc][s]:list = list()
            for route in routes[s]:
                total_serviced_costumers[sc][s].extend(route)
                total_total.extend(route)
        
        print(f'Initial {sc} scenario done')
        return (sc, routes, total_serviced_costumers, total_total)

    lab = pool.Pool(                                             )
    Results = lab.map(generate_routes_for_scenario, range(25))

    return Results

In [3]:
# Auxiliary functions - Clusters
def compute_shared_vehicles(S, K_s): 
    shared_vehicles = dict()
    for s in S:
        for ss in S:
            if s != ss:
                shared_vehicles[s,ss] = sum(len(set(K_s[sc][s]).intersection(set(K_s[sc][ss]))) for sc in range(25))/25
                # shared_vehicles[s,ss] = sum([1 for sc in range(25) for k in K_s[sc][s] if k in K_s[sc][ss]])/25
    return shared_vehicles


def get_associated_stations(S, K_s, station):
    # Get average number of shared vehicle per station
    shared_vehicles = compute_shared_vehicles(S, K_s)

    # Filter the dictionary to get the key-value pairs with matching i
    filtered_dict = {key: value for key, value in shared_vehicles.items() if key[0] == station}
    
    # Sort the filtered dictionary by values in descending order
    sorted_dict = sorted(filtered_dict.items(), key=lambda x: x[1], reverse=True)
    
    # Extract the top four j values from the sorted dictionary
    top_j_values = [key[1] for key, value in sorted_dict[:4]]

    # Locate vehicles assigned to the stations
    K = dict()
    for sc in range(25):
        K[sc] = list()
        for s in [station]+top_j_values:
            for k in K_s[sc][s]:
                if k not in K[sc]:
                    K[sc].append(k)

    return shared_vehicles, top_j_values, K

# Greedy: Give chargers to stations given their stress index (individually)

In [4]:
start = process_time()

ordered_stations:list = sorted(original_stress, key = original_stress.get, reverse = True)     # Get list of stations ordered by stress
stress = deepcopy(original_stress)

y:dict = dict(); y.update({s:0 for s in S})
Results = dict()
Results.update({sc:{'routes': dict(), 'total_serviced_costumers': dict(), 'total_total':list()} for sc in range(25)})

K_prime = deepcopy(K)
K_s_prime = deepcopy(K_s) 

cont:int = 0

while True:
    cont += 1 
    s = ordered_stations[0]
    y[s] = 1       # Start giving one charger to the station

    # Iterative process to find number of chargers 
    while True:
        routes, total_serviced_costumers = generate_routes_given_number_chargers_individual(s, S, y[s], K_prime, K_s_prime)     # Get the routes for the stations
        
        if routes == None:  # Station doesn't service all possible costumers in at least one scenario
            y[s] += 1
        else:               # Number of chargers is fixed
            break

    if cont == 1: print(f'Iter \tSt \t#ch \tStress \tProgr \tTime \n------------------------------------------------------')
    print(f'{cont} \t{s} \t{y[s]} \t{round(stress[s],2)} \t{round(round(cont/600,2)*100,4)}% \t{round(process_time()-start,2)}')    

    # Save routes and remove serviced vehicles
    for sc in range(25):
        Results[sc]['routes'][s] = routes[sc]
        Results[sc]['total_serviced_costumers'][s] = total_serviced_costumers[sc]

        for vehicle in Results[sc]['total_serviced_costumers'][s]:
            Results[sc]['total_total'].append(vehicle)
            K_prime[sc].remove(vehicle)
        
            for ss in S:
                if vehicle in K_s_prime[sc][ss]:
                    K_s_prime[sc][ss].remove(vehicle)

    S.remove(s)

    # Recompute stress index
    stress:dict = {}
    for s in S:
        stress[s] = np.mean(np.array([len(K_s_prime[sc][s]) for sc in range(25)]))    
    ordered_stations:list = sorted(stress, key = stress.get, reverse = True)     # Get list of stations ordered by stress

    # Stoping condition: All stations are assigned or no vehicles are left on any scenario
    if cont == 600 or sum(len(K_prime[sc]) for sc in range(25)) == 0:
        break

Results['Chargers'] = y

a_file = open(path+'/Results/Number of chargers/Greedy assignment', "wb")
pickle.dump(Results, a_file)
a_file.close()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-07


c:\Users\a.rojasa55\OneDrive - Universidad de los andes\Documentos\MOPTA-23\source.py:212: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  cK.update({k:1 if k in li else 0 for k in cK})


Iter 	St 	#ch 	Stress 	Progr 	Time 
------------------------------------------------------
1 	459 	8 	197.0 	0.0% 	24.2
2 	609 	8 	196.16 	0.0% 	51.77
3 	1154 	8 	169.0 	1.0% 	68.56
4 	207 	8 	153.92 	1.0% 	87.02
5 	808 	8 	148.44 	1.0% 	106.81
6 	113 	8 	133.68 	1.0% 	131.36
7 	46 	8 	113.8 	1.0% 	145.27
8 	1033 	8 	113.64 	1.0% 	156.48
9 	768 	8 	105.84 	1.0% 	167.16
10 	746 	8 	101.52 	2.0% 	177.25
11 	170 	8 	99.88 	2.0% 	189.53
12 	1047 	8 	96.76 	2.0% 	204.56
13 	1088 	8 	94.52 	2.0% 	215.66
14 	716 	8 	92.52 	2.0% 	223.91
15 	437 	8 	92.36 	3.0% 	234.52
16 	356 	8 	87.96 	3.0% 	246.03
17 	767 	8 	86.56 	3.0% 	256.84
18 	889 	8 	84.48 	3.0% 	264.2
19 	279 	8 	79.12 	3.0% 	272.11
20 	379 	8 	76.16 	3.0% 	278.94
21 	886 	8 	75.52 	4.0% 	286.86
22 	290 	8 	73.16 	4.0% 	294.03
23 	1032 	8 	70.96 	4.0% 	302.25
24 	664 	8 	67.28 	4.0% 	311.73
25 	753 	8 	67.24 	4.0% 	318.64
26 	1077 	8 	65.92 	4.0% 	325.97
27 	438 	8 	65.4 	4.0% 	332.17
28 	268 	8 	64.08 	5.0% 	340.03
29 	556 	8 	63.12

In [9]:
n = Results["Chargers"]
n = {s:n[s] for s in n if n[s] > 0}
S = list(n.keys())
len(S)

file = open(path+f"/Results/Optimal/S","wb")
pickle.dump(S,file); file.close()

file = open(path+f"/Results/Optimal/n","wb")
pickle.dump(n,file); file.close()

file = open(path+f"/Results/Optimal/results","wb")
pickle.dump(Results,file); file.close()


In [10]:

def generate_distances(vehicles,stations,S):
    distances = dict()
    for i in vehicles.index:
        for st in S:    
            distances[st,i] = euclidean([vehicles["0"][i], vehicles["1"][i]],[stations["0"][st], stations["1"][st]])
    return distances

stations = pd.read_csv(path+"/Data/Definite Stations.csv",index_col=[0])
vehicles = pd.read_csv(path+"/Data/Definite Vehicles.csv",index_col=[0])

d_matrix = generate_distances(vehicles, stations, S)

file = open(path+f"/Results/Optimal/distances","wb")
pickle.dump(d_matrix,file); file.close()

# Give chargers to stations given their dual variables (cluster)

In [ ]:
ordered_stations:list = sorted(original_stress, key = original_stress.get, reverse = True)     # Get list of stations ordered by stress
number_of_chargers:dict = dict()
stress = deepcopy(original_stress)

K_prime = deepcopy(K)
K_s_prime = deepcopy(K_s) 

cont:int = 0

while True:
    cont += 1
    s = ordered_stations[0]
    shared_vehicles, compatible_stations, K_prime = get_associated_stations(S, K_s_prime, s)
    print(f'Cluster # {cont}: {compatible_stations}') 
    
    y:dict = {st:1 for st in compatible_stations}       # Start giving one charger to the station

    # Iterative process to find number of chargers 
    while True:
        print(f"\tTrying for {list(y.values())} configuration")
        total_serviced_costumers, sigmas, missing = generate_routes_given_number_chargers(compatible_stations, y, K_prime, K_s_prime, False)     # Get the routes for the stations
        
        if missing:
            sigmas = {key:value for key,value in sigmas.items() if y[key]<8}
            best_station = min(sigmas, key = sigmas.get)
            y[best_station] += 1
        else:
            break
    
    # Set number of chargers and remove already serviced vehicles
    for st in compatible_stations:
        number_of_chargers[st] = y[st]
        print(f'\n\tStation {st} set to {number_of_chargers[st]} chargers\nn----------------------------')

        for u in range(25):
            for vehicle in total_serviced_costumers[u][st]:
                K_prime[u].remove(vehicle)
            
                for ss in S:
                    if vehicle in K_s_prime[u][ss]:
                        K_s_prime[u][ss].remove(vehicle)

        S.remove(st)

    stress:dict = {}
    for s in S:
        stress[s] = np.mean(np.array([len(K_s_prime[sc][s]) for sc in range(25)]))    
    ordered_stations:list = sorted(stress, key = stress.get, reverse = True)     # Get list of stations ordered by stress

    if cont == 600 or sum(len(K_prime[sc]) for sc in range(25))==0:
        break

# Two phases: Give one charger to each station and then assing given their stress index

In [5]:
start = process_time()
initial_routes = 'load' #'generate'

# Duplicate parameters
K_prime = deepcopy(K)
K_s_prime = deepcopy(K_s) 
ordered_stations:list = sorted(original_stress, key = original_stress.get, reverse = True)     # Get list of stations ordered by stress

# Give one charger to each station
y:dict = dict(); y.update({s:1 for s in S})

# Generate initial routes (one charger per station)
if initial_routes == 'generate': 
    Results = generate_initial_routes(S, y, K_prime, K_s_prime, False)
    print(f'\n\t--------- Finished initial assignment at {round(process_time()-start,s)} --------- \n')
elif initial_routes == 'load': 
    file = open(path+'/Results/Number of chargers/Initial Routes', 'rb'); Results = pickle.load(file); file.close()
    print(f'\n\t--------- Loaded initial assignment --------- \n')


Results = {i[0]:{'routes':i[1], 'total_serviced_costumers':i[2][i[0]], 'total_total':i[3]} for i in Results}
for sc in range(25):
        K, K_s, S_k, a, t = load_pickle(path+"/Data/",sc)
        for s in S:
            if len(Results[sc]['routes'][s]) != 0:
                Results[sc]['routes'][s] = [sorted(Results[sc]['routes'][s][0] , key = lambda ii: a.get((ii,s)))]

# Fix serviced vehicles per each station
for sc, values in Results.items():
    for vehicle in values['total_total']:
        K_prime[sc].remove(vehicle)
        for s in S:
            if vehicle in K_s_prime[sc][s]:
                K_s_prime[sc][s].remove(vehicle)
                

# If a station no longer has availabe vehicles on any scenario, remove. Else, compute new stress index
stress:dict = dict()
for s in S:
    if sum(len(K_s_prime[sc][s]) for sc in range(25))==0:
        S.remove(s)
    else:
        stress[s] = np.mean(np.array([len(K_s_prime[sc][s]) for sc in range(25)]))    

ordered_stations:list = sorted(stress, key = stress.get, reverse = True)     # Get list of stations ordered by stress

# Iterative process
cont:int = 600-len(S)
second_cont = 0
while True:
    cont += 1
    second_cont += 1
    s = ordered_stations[0]

    y_aux = 1

    # Iterative process to find number of chargers 
    while True:
        routes, total_serviced_costumers = generate_routes_given_number_chargers_individual(s, S, y_aux, K_prime, K_s_prime, 7)     # Get the routes for the stations
        
        if routes == None:
            y_aux += 1
        else: 
            break
    
    if second_cont == 1: print(f'Iter \tSt \t#ch \tStress \tProgr \tTime \n------------------------------------------------------')
    y[s] += y_aux
    print(f'{second_cont} \t{s} \t{y[s]} \t{round(stress[s],2)} \t{round(round(cont/600,2)*100,4)}% \t{round(process_time()-start,2)}')    

    # Save routes and remove serviced vehicles
    for sc in range(25):
        rr = routes[sc]
        Results[sc]['routes'][s].extend(rr)

        for vehicle in total_serviced_costumers[sc]:
            Results[sc]['total_serviced_costumers'][s].append(vehicle)
            Results[sc]['total_total'].append(vehicle)

            K_prime[sc].remove(vehicle)
        
            for ss in S:
                if vehicle in K_s_prime[sc][ss]:
                    K_s_prime[sc][ss].remove(vehicle)

    S.remove(s)

    # Recompute stress index
    stress:dict = dict()
    for s in S:
        stress[s] = np.mean(np.array([len(K_s_prime[sc][s]) for sc in range(25)]))    
    ordered_stations:list = sorted(stress, key = stress.get, reverse = True)     # Get list of stations ordered by stress

    # Stoping condition: All stations are assigned or no vehicles are left on any scenario
    if cont == 600 or sum(len(K_prime[sc]) for sc in range(25)) == 0:
        break

Results['Chargers'] = y

a_file = open(path+'/Results/Number of chargers/Two phase assignment', "wb")
pickle.dump(Results, a_file)
a_file.close()


	--------- Loaded initial assignment --------- 

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-11


/Users/juanbeta/My Drive/Research/MOPTA/MOPTA-23/source.py:212: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  cK.update({k:1 if k in li else 0 for k in cK})


Iter 	St 	#ch 	Stress 	Progr 	Time 
------------------------------------------------------
1 	379 	8 	125.72 	2.0% 	43.15
2 	889 	8 	84.32 	2.0% 	56.86
3 	1124 	8 	82.24 	2.0% 	68.65
4 	207 	8 	77.52 	3.0% 	80.5
5 	808 	8 	76.84 	3.0% 	90.31
6 	113 	8 	48.36 	3.0% 	101.56
7 	373 	8 	45.88 	3.0% 	108.32
8 	405 	8 	43.32 	3.0% 	115.56
9 	1033 	7 	43.32 	3.0% 	125.0
10 	46 	7 	40.44 	4.0% 	131.23
11 	217 	8 	38.44 	4.0% 	140.33
12 	951 	6 	36.76 	4.0% 	146.86
13 	279 	8 	35.64 	4.0% 	152.96
14 	19 	8 	31.92 	4.0% 	159.21
15 	408 	8 	30.48 	4.0% 	164.07
16 	290 	7 	28.44 	4.0% 	169.08
17 	424 	8 	27.84 	5.0% 	174.66
18 	1047 	7 	27.72 	5.0% 	181.53
19 	10 	7 	27.48 	5.0% 	190.24
20 	340 	6 	27.48 	5.0% 	197.58
21 	1135 	6 	26.64 	5.0% 	201.74
22 	13 	6 	25.56 	6.0% 	206.0
23 	885 	6 	23.52 	6.0% 	211.82
24 	318 	8 	23.12 	6.0% 	215.7
25 	193 	5 	20.68 	6.0% 	218.77
26 	1052 	5 	19.84 	6.0% 	221.81
27 	1146 	5 	19.8 	6.0% 	226.08
28 	446 	5 	19.72 	7.0% 	230.5
29 	1216 	5 	19.48 	7.0% 	233.

# Retrieving information

In [2]:
# Auxiliary functions - Upload and display result

# Retrieve results
def upload_data(heuristic):
    file = open(path + f'/Results/Number of chargers/{heuristic}', 'rb')
    Results = pickle.load(file)
    file.close()

    return Results

# Print stats of solution
def print_general_information(heuristic, Results):
    print(f'---------------- Heuristic {heuristic} ----------------')
    print(f'- Number of active stations: {sum([1 for value in Results["Chargers"].values() if value != 0])}')
    print(f'- Number of chargers: {sum(Results["Chargers"].values())} \n')

# Export Results
def export_num_chargers(Results_greedy, Results_two_phase):
    data = dict(); data.update({'station':S})

    data['greedy_chargers'] = list();   data['two_phase_chargers'] = list();   data['stress'] = list()

    for s in S:
        data['greedy_chargers'].append(Results_greedy['Chargers'][s])
        data['two_phase_chargers'].append(Results_two_phase['Chargers'][s])
        data['stress'].append(original_stress[s])
    

    df = pd.DataFrame(data = data)
    df.to_excel(path+ f'/Results/Results.xlsx')


def generate_distances():
    stations = pd.read_csv(path+"/Data/Definite Stations.csv",index_col=[0])
    vehicles = pd.read_csv(path+"/Data/Definite Vehicles.csv",index_col=[0])

    distances = dict()
    for i in vehicles.index:
        for st in stations.index:    
            distances[st,i] = euclidean([vehicles["0"][i], vehicles["1"][i]],[stations["0"][st], stations["1"][st]])
    return distances


def reconstruct_routes(heuristics):
    # Reconstructing the routes
    schedules = {idx:{sc:{} for sc in range(25)} for idx,_ in enumerate(heuristics)}

    active_stations = {idx:{sc:0 for sc in range(25)} for idx in range(len(heuristics))}
    avg_distance = {idx:{sc:0 for sc in range(25)} for idx in range(len(heuristics))}
    visited = {idx:{sc:[] for sc in range(25)} for idx in range(len(heuristics))}

    avg_charging_time = {idx:{sc:0 for sc in range(25)} for idx in range(len(heuristics))}

    avg_utilization = {idx:{sc:0 for sc in range(25)} for idx in range(len(heuristics))}
    active_chargers = {idx:{sc:0 for sc in range(25)} for idx in range(len(heuristics))}

    avg_waiting_time = {idx:{sc:0 for sc in range(25)} for idx in range(len(heuristics))}

    avg_driving_charging_cost = {idx:{sc:0 for sc in range(25)} for idx in range(len(heuristics))}
    latest_finalization = {idx:{sc:0 for sc in range(25)} for idx in range(len(heuristics))}

    for idx, heuristic in enumerate(heuristics):
        idx2 = 0
        if heuristic != 'Two phase assignment':    
            idx2 = 1
            
        for s in S:
            charger_num = Results[idx2][f'Chargers'][s]
            
            # Station has no chargers assigned
            if charger_num == 0: 
                continue

            # Retrieve information from all scenarios
            for sc in range(25):
                station_is_active = False
                K, K_s, S_k, a, t = load_pickle(path+"/Data/",sc)
                file = open(f'{path}/Data/p/p_{sc}','rb'); p = pickle.load(file); file.close()
                active_char = 0
                
                routes = Results[idx2][sc]['routes'][s]

                schedules[idx][sc][s] = list()

                # Station has no assigned vehicles that scenario (unactive)
                if len(routes) == 0: 
                    continue
                
                station_is_active = True
                active_chargers[idx][sc] += Results[idx2]['Chargers'][s]

                for route in routes:
                    schedule = {'vehicles':list(), 'arrive':list(),'wait':list(), 'start':list(), 'end':list()}

                    for pos, vehicle in enumerate(route):
                        visited[idx][sc].append(vehicle)
                        avg_distance[idx][sc] += distances[s,vehicle]/len(Results[idx2][sc]['total_total'])
                        schedule['vehicles'].append(vehicle)

                        if pos == 0:
                            # print(sc,vehicle,s)
                            schedule['arrive'].append(a[vehicle,s])
                            schedule['start'].append(a[vehicle,s])
                            schedule['wait'].append(0)
                            schedule['end'].append(a[vehicle,s] + t[vehicle,s])
                        else:
                            schedule['arrive'].append(a[vehicle,s])

                            if schedule['end'][-1] > a[vehicle,s]:
                                schedule['wait'].append(schedule['end'][-1] - a[vehicle,s])
                            else:
                                schedule['wait'].append(0)
                            avg_waiting_time[idx][sc] += schedule['wait'][-1]/len(Results[idx2][sc]['total_total'])

                            schedule['start'].append(schedule['arrive'][-1] + schedule['wait'][-1])
                            schedule['end'].append(schedule['start'][-1] + t[vehicle,s])

                        avg_charging_time[idx][sc] += (schedule['end'][-1] - schedule['start'][-1])/len(Results[idx2][sc]['total_total'])
                        avg_utilization[idx][sc] += schedule['end'][-1] - schedule['start'][-1]

                        avg_driving_charging_cost[idx][sc] += p[vehicle,s] * 0.0388  + distances[s,vehicle] * 0.041
                    
                    schedules[idx][sc][s].append(schedule)

                if station_is_active: 
                    active_stations[idx][sc] += 1
                 

                latest_finalization[idx][sc] = max(schedule['end'][-1], 14)

    for sc in range(25):
        avg_utilization[idx][sc] = (avg_utilization[idx][sc] / active_chargers[idx][sc])

    return schedules, active_stations, avg_distance, visited, avg_charging_time, avg_utilization, avg_waiting_time, avg_driving_charging_cost, latest_finalization


def weight_scenarios(iterable):
    return round(sum(iterable[sc] for sc in range(25))/25,2)


def compute_interval(iterable):
    # Compute mean and standard deviation
    mean = np.mean(iterable)
    std_dev = np.std(iterable)

    # Degrees of freedom
    df = len(iterable) - 1

    # Alpha level (e.g., 0.05 for 95% confidence interval)
    alpha = 0.025

    # Calculate t-value
    t_value = stats.t.ppf(1 - alpha/2, df)

    # Calculate standard error
    std_err = std_dev / np.sqrt(len(iterable))

    # Calculate confidence interval
    lower_bound = mean - t_value * std_err
    upper_bound = mean + t_value * std_err

    return round(lower_bound,2), round(upper_bound,2)

# Load the pickled distances
file = open('Data/distance_matrix', 'rb')
distances = pickle.load(file)
file.close()

heuristics = ['Two phase assignment', 'Greedy assignment']

Results_two_phase = upload_data(heuristics[0])                  # Load data
Results_greedy = upload_data(heuristics[1])
Results = [Results_two_phase, Results_greedy]

two_phases_number_of_chargers = sum(Results_two_phase['Chargers'].values())

greedy_number_of_chargers = sum(Results_greedy['Chargers'].values())
greedy_number_of_stations = sum(1 for i,j in Results_greedy['Chargers'].items() if j > 0)

In [3]:
heuristics = ['Two phase assignment']
print_general_information(heuristics[0], Results_two_phase)

### Reconstructing the routes
schedules, active_stations, avg_distance, visited, avg_charging_time, avg_utilization, avg_waiting_time, avg_driving_charging_cost, latest_finalization = reconstruct_routes(heuristics)

---------------- Heuristic Two phase assignment ----------------
- Number of active stations: 600
- Number of chargers: 984 



In [4]:
heuristics = ['Greedy assignment']
print_general_information(heuristics[0], Results_greedy)

schedules_2, active_stations_2, avg_distance_2, visited_2, avg_charging_time_2, avg_utilization_2, avg_waiting_time_2, avg_driving_charging_cost_2, latest_finalization_2 = reconstruct_routes(heuristics)

---------------- Heuristic Greedy assignment ----------------
- Number of active stations: 165
- Number of chargers: 744 



## Printing results

In [5]:
heuristics = ['Two phase assignment', 'Greedy assignment']

two_phases_number_of_chargers = sum(Results_two_phase['Chargers'].values())

greedy_number_of_chargers = sum(Results_greedy['Chargers'].values())
greedy_number_of_stations = sum(1 for i,j in Results_greedy['Chargers'].items() if j > 0)

SC = range(25)

# print(f'\t \t \t{heuristics[0][:5]}. \t{heuristics[1][:6]}')
# print(f'Opended stations \t{600} \t{greedy_number_of_stations}')
# print(f'Chargers \t\t{two_phases_number_of_chargers} \t{greedy_number_of_chargers}')
# print(f'Avg active stations \t{weight_scenarios(active_stations[0])}\t{weight_scenarios(active_stations_2[0])}')
# print(f'Avg stress index \t{round(sum(original_stress.values())/600,2)} \t{round(sum(j for i,j in original_stress.items() if i in Results[1][0]["routes"].keys())/greedy_number_of_stations,2)}')
# print(f'Avg serviced car \t{round(sum(len(K[sc])  for sc in range(25))/(25*600),2)} \t{round(sum(len(K[sc])  for sc in range(25))/(25 * greedy_number_of_stations),2)}')
# print(f'Avg charging time \t{weight_scenarios(avg_charging_time[0])}\t{weight_scenarios(avg_charging_time_2[0])}')
# print(f'Avg traveled distance \t{weight_scenarios(avg_distance[0])}\t{weight_scenarios(avg_distance_2[0])}')

# print(f'Avg Utilization: \t{round(sum(avg_utilization[0][sc]/latest_finalization[0][sc] for sc in range(25))/25,2)} \t{round(sum(avg_utilization_2[0][sc]/latest_finalization_2[0][sc] for sc in range(25))/25,2)}')

# print(f'Avg waiting time \t{weight_scenarios(avg_waiting_time[0])}\t{weight_scenarios(avg_waiting_time_2[0])}')
# print(f'Expected cost \t\t{weight_scenarios(avg_driving_charging_cost[0])} {weight_scenarios(avg_driving_charging_cost_2[0])}')

print('--------------------------- Two phases -----------------------------')
print(f'\t \t \t{heuristics[0][:6]} \tmin \tlow \thigh \tmax')
print(f'Opended stations \t{600} \t- \t- \t- \t-')
print(f'Chargers \t \t{two_phases_number_of_chargers} \t- \t- \t- \t-')
print(f'Avg stress index  \t{round(sum(j for i,j in original_stress.items() if i in Results[1][0]["routes"].keys())/greedy_number_of_stations,2)} \t- \t- \t- \t-')

val = active_stations[0]; lower_bound, upper_bound = compute_interval(list(val.values()))
print(f'Avg active stations \t{weight_scenarios(val)} \t{min(val.values())} \t{lower_bound} \t{upper_bound} \t{max(val.values())}')

val = [len(K[sc])/600 for sc in SC]; lower_bound, upper_bound = compute_interval(val)
print(f'Avg serviced car \t{round(sum(val)/25,2)} \t{round(min(val),2)} \t{lower_bound} \t{upper_bound} \t{round(max(val),2)}')

val = avg_charging_time[0]; lower_bound, upper_bound = compute_interval(list(val.values()))
print(f'Avg charging time \t{weight_scenarios(val)} \t{round(min(val.values()),2)} \t{lower_bound} \t{upper_bound}\t{round(max(val.values()),2)}')

val = avg_distance[0]; lower_bound, upper_bound = compute_interval(list(val.values()))
print(f'Avg traveled distance \t{weight_scenarios(val)} \t{round(min(val.values()),2)}  \t{lower_bound} \t{upper_bound} \t{round(max(val.values()),2)}')

val = [avg_utilization[0][i]/latest_finalization[0][i] for i in SC]; lower_bound, upper_bound = compute_interval(val)
print(f'Avg Utilization: \t{round(sum(val)/25,2)} \t{round(min(val),2)}  \t{lower_bound} \t{upper_bound} \t{round(max(val),2)}')

val = avg_waiting_time[0]; lower_bound, upper_bound = compute_interval(list(val.values()))
print(f'Avg waiting time \t{weight_scenarios(val)}\t{round(min(val.values()),2)}  \t{lower_bound} \t{upper_bound} \t{round(max(val.values()),2)}')

val = avg_driving_charging_cost[0]; lower_bound, upper_bound = compute_interval(list(val.values()))
print(f'Expected cost \t\t{round(weight_scenarios(val),1)} {round(min(val.values()),1)}  {round(lower_bound,1)} {round(upper_bound,1)} {round(max(val.values()),1)}')



print('\n \n--------------------------- Greedy -----------------------------')
print(f'\t \t \t{heuristics[1][:6]} \tmin \tlow \thigh \tmax')
print(f'Opended stations \t{greedy_number_of_stations} \t- \t- \t- \t-')
print(f'Chargers \t \t{greedy_number_of_chargers} \t- \t- \t- \t-')
print(f'Avg stress index  \t{round(sum(j for i,j in original_stress.items() if i in Results[1][0]["routes"].keys())/greedy_number_of_stations,2)} \t- \t- \t- \t-')

val = active_stations_2[0]; lower_bound, upper_bound = compute_interval(list(val.values()))
print(f'Avg active stations \t{weight_scenarios(val)} \t{min(val.values())} \t{lower_bound} \t{upper_bound} \t{max(val.values())}')

val = [len(K[sc])/greedy_number_of_stations for sc in SC]; lower_bound, upper_bound = compute_interval(val)
print(f'Avg serviced car \t{round(sum(val)/25,2)} \t{round(min(val),2)} \t{lower_bound} \t{upper_bound} \t{round(max(val),2)}')

val = avg_charging_time_2[0]; lower_bound, upper_bound = compute_interval(list(val.values()))
print(f'Avg charging time \t{weight_scenarios(val)} \t{round(min(val.values()),2)} \t{lower_bound} \t{upper_bound}\t{round(max(val.values()),2)}')

val = avg_distance_2[0]; lower_bound, upper_bound = compute_interval(list(val.values()))
print(f'Avg traveled distance \t{weight_scenarios(val)} \t{round(min(val.values()),2)}  \t{lower_bound} \t{upper_bound} \t{round(max(val.values()),2)}')

val = [avg_utilization_2[0][i]/latest_finalization_2[0][i] for i in SC]; lower_bound, upper_bound = compute_interval(val)
print(f'Avg Utilization: \t{round(sum(val)/25,2)} \t{round(min(val),2)}  \t{lower_bound} \t{upper_bound} \t{round(max(val),2)}')

val = avg_waiting_time_2[0]; lower_bound, upper_bound = compute_interval(list(val.values()))
print(f'Avg waiting time \t{weight_scenarios(val)}\t{round(min(val.values()),2)}  \t{lower_bound} \t{upper_bound} \t{round(max(val.values()),2)}')

val = avg_driving_charging_cost_2[0]; lower_bound, upper_bound = compute_interval(list(val.values()))
print(f'Expected cost \t\t{round(weight_scenarios(val),1)} {round(min(val.values()),1)}  {round(lower_bound,1)} {round(upper_bound,1)} {round(max(val.values()),1)}')

--------------------------- Two phases -----------------------------
	 	 	Two ph 	min 	low 	high 	max
Opended stations 	600 	- 	- 	- 	-
Chargers 	 	984 	- 	- 	- 	-
Avg stress index  	63.03 	- 	- 	- 	-
Avg active stations 	459.36 	442 	456.17 	462.55 	474
Avg serviced car 	7.54 	7.35 	7.5 	7.59 	7.82
Avg charging time 	1.17 	1.16 	1.17 	1.17	1.18
Avg traveled distance 	5.96 	5.81  	5.92 	5.99 	6.09
Avg Utilization: 	0.45 	0.41  	0.44 	0.45 	0.47
Avg waiting time 	0.35	0.34  	0.35 	0.36 	0.36
Expected cost 		32929.5 32051.3  32723.8 33135.1 34172.7

 
--------------------------- Greedy -----------------------------
	 	 	Greedy 	min 	low 	high 	max
Opended stations 	165 	- 	- 	- 	-
Chargers 	 	744 	- 	- 	- 	-
Avg stress index  	63.03 	- 	- 	- 	-
Avg active stations 	143.52 	139 	142.29 	144.75 	148
Avg serviced car 	27.43 	26.73 	27.27 	27.59 	28.42
Avg charging time 	1.17 	1.16 	1.17 	1.17	1.18
Avg traveled distance 	5.86 	5.74  	5.83 	5.89 	5.96
Avg Utilization: 	0.52 	0.47  	0.51 	0.53

In [7]:
sum(1 for i,j in Results[0]['Chargers'].items() if j == 1)

{0: {0: 14,
  1: 14,
  2: 14,
  3: 14.442835696820357,
  4: 14.578687430089149,
  5: 14,
  6: 14,
  7: 14,
  8: 14,
  9: 15.338647435506122,
  10: 14,
  11: 15.208111812532264,
  12: 14.298268715880798,
  13: 14,
  14: 14,
  15: 14,
  16: 14,
  17: 14,
  18: 14.406124500018924,
  19: 14,
  20: 14.640588996612466,
  21: 15.170594835391787,
  22: 14,
  23: 14,
  24: 14}}

In [12]:
avg_utilization[0]

{0: 6.479279101134932,
 1: 6.282554196997305,
 2: 6.12940992231145,
 3: 6.336498168353987,
 4: 6.395092617244196,
 5: 6.531494991630828,
 6: 6.632018233609541,
 7: 6.450740889460349,
 8: 6.237021525581904,
 9: 6.299570030238833,
 10: 6.210568078687018,
 11: 6.358147439313253,
 12: 6.377001528631364,
 13: 6.325031241317549,
 14: 6.330716919787156,
 15: 6.379157391077116,
 16: 6.392385392129397,
 17: 6.331246176129525,
 18: 6.344076481071716,
 19: 6.260053652557951,
 20: 6.334324818949835,
 21: 6.296334901593829,
 22: 6.247498584219628,
 23: 6.3468505987715265,
 24: 6.334020734871053}

In [5]:
start = process_time()
initial_routes = 'load' #'generate'

# Duplicate parameters
K_prime = deepcopy(K)
K_s_prime = deepcopy(K_s) 
ordered_stations:list = sorted(original_stress, key = original_stress.get, reverse = True)     # Get list of stations ordered by stress

# Give one charger to each station
y:dict = dict(); y.update({s:1 for s in S})

# Generate initial routes (one charger per station)
if initial_routes == 'generate': 
    Results = generate_initial_routes(S, y, K_prime, K_s_prime, False)
    print(f'\n\t--------- Finished initial assignment at {round(process_time()-start,s)} --------- \n')
elif initial_routes == 'load': 
    file = open(path+'/Results/Number of chargers/Initial Routes', 'rb'); Results = pickle.load(file); file.close()
    print(f'\n\t--------- Loaded initial assignment --------- \n')


Results = {i[0]:{'routes':i[1], 'total_serviced_costumers':i[2][i[0]], 'total_total':i[3]} for i in Results}
for sc in range(25):
        K, K_s, S_k, a, t = load_pickle(path+"/Data/",sc)
        for s in S:
            print(s)
            if len(Results[sc]['routes'][s]) != 0:
                Results[sc]['routes'][s] = [sorted(Results[sc]['routes'][s][0] , key = lambda ii: a.get((ii,s)))]
                    





	--------- Loaded initial assignment --------- 

2
[[112, 123, 1201, 1270, 3351, 3473, 4339, 4416, 5440, 5509, 5586, 6529, 8692, 8721, 8857, 9823]]
4
5
6
[[458, 1405, 2499, 2612, 5674, 5875, 6969, 7128, 7894, 8973, 9127, 9164, 10321]]
7
[[2192, 3242, 5616, 6508]]
8
[[1927, 5094, 6145, 6259, 6315, 7241, 10583, 10596]]
9
[[268, 2754, 4558, 4584, 8149, 8874, 9979, 10025, 10409]]
10
[[244, 536, 1339, 2524, 2822, 4753, 4860, 5655, 5829, 6806, 6911, 8904, 9983]]
11
[[608, 7947, 8261]]
12
[[1831, 2971, 3074, 7226, 7390, 8366, 10458]]
13
[[706, 1547, 1725, 2516, 3848, 5053, 7003, 7167, 8929, 10273, 10357]]
14
[[1406, 2460]]
15
[[2366, 3317, 5515, 6625, 8840, 9831]]
16
[[8353, 8442]]
17
[[1075, 1078, 2154, 6467]]
19
[[1410, 1702, 2849, 3572, 4607, 5007, 7888, 9255, 10046, 10401]]
20
21
22
[[1517, 2586, 2751, 4754, 4917, 6997, 7035, 7067, 7075, 8947, 10226]]
23
[[1033, 5331, 8560, 8584]]
24
[[2189, 2264, 4333, 4459, 4508, 5555, 6676, 7745, 8640, 8727]]
25
[[133, 6704, 7686, 7783, 8862]]
26
27
[